In [1]:
import pandas as pd
import glob
import seaborn as sns

folder = "/workspaces/ecommerce_analysis/data/stage=raw/source=search/dataformat=parquet"
# written_files = glob.glob(f'{folder}/*')

# for file in written_files:
#     pdf = pd.read_parquet(file, engine="pyarrow")

pdf = pd.read_parquet(folder, engine="pyarrow")

with pd.option_context('display.max_columns', None,):
    display(pdf.head())


,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,thumbnail,currency_id,order_backend,price,original_price,sale_price,sold_quantity,available_quantity,official_store_id,official_store_name,use_thumbnail_id,accepts_mercadopago,tags,shipping,stop_time,seller,seller_address,address,attributes,installments,winner_item_id,catalog_listing,discounts,promotions,inventory_id,differential_pricing
0,MLA1240290150,Smart Tv Noblex Dq50x9500pi Qled 4k 50'' Black...,new,623602-MLA52079115465_102022,MLA19712052,gold_special,https://www.mercadolibre.com.ar/smart-tv-noble...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_623602-MLA52079115...,ARS,1,149499.0,219999.0,None,100,250,1192.0,Noblex,True,True,"[good_quality_picture, good_quality_thumbnail,...","{'free_shipping': True, 'logistic_type': 'cros...",2043-04-15T20:39:11.000Z,"{'_': False, 'car_dealer': False, 'car_dealer_...","{'address_line': '', 'city': {'id': None, 'nam...","{'city_id': None, 'city_name': 'Monte Chingolo...","[{'attribute_group_id': 'MAIN', 'attribute_gro...","{'amount': 27035.65, 'currency_id': 'ARS', 'qu...",None,True,None,[],None,None
1,MLA1236126329,Smart Tv Noblex Db58x7500 58 Led 4k Android Tv,new,970198-MLA51856807153_102022,MLA19693852,gold_pro,https://www.mercadolibre.com.ar/smart-tv-noble...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_970198-MLA51856807...,ARS,2,156999.0,196999.0,None,100,150,1255.0,Castillo,True,True,"[extended_warranty_eligible, good_quality_pict...","{'free_shipping': False, 'logistic_type': 'def...",2042-11-06T04:00:00.000Z,"{'_': False, 'car_dealer': False, 'car_dealer_...","{'address_line': '', 'city': {'id': 'TUxBQ1RBR...","{'city_id': 'TUxBQ1RBRjFlMTNk', 'city_name': '...","[{'attribute_group_id': 'MAIN', 'attribute_gro...","{'amount': 26166.5, 'currency_id': 'ARS', 'qua...",None,True,None,[],None,{'id': 35713325.0}
2,MLA1156297648,Smart Tv Tcl C72-series 50c725 Qled Android Pi...,new,613322-MLA53779631053_022023,MLA18597975,gold_special,https://www.mercadolibre.com.ar/smart-tv-tcl-c...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_613322-MLA53779631...,ARS,3,147399.0,NaN,None,5,1,1425.0,Diggit,True,True,"[good_quality_thumbnail, standard_price_by_cha...","{'free_shipping': True, 'logistic_type': 'fulf...",2042-09-21T18:15:41.000Z,"{'_': False, 'car_dealer': False, 'car_dealer_...","{'address_line': '', 'city': {'id': None, 'nam...","{'city_id': None, 'city_name': 'Garin', 'state...","[{'attribute_group_id': 'MAIN', 'attribute_gro...","{'amount': 26655.88, 'currency_id': 'ARS', 'qu...",None,True,None,[],NZKJ32770,None
3,MLA1148941757,Smart Tv Samsung Series 7 Un55au7000gczb Led T...,new,739692-MLA48913871021_012022,MLA18718085,gold_pro,https://www.mercadolibre.com.ar/smart-tv-samsu...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_739692-MLA48913871...,ARS,4,175999.0,219999.0,None,500,250,1938.0,Mercado Libre Electronica,True,True,"[good_quality_thumbnail, ahora-6, bulky, exten...","{'free_shipping': True, 'logistic_type': 'fulf...",2042-11-25T18:00:04.000Z,"{'_': False, 'car_dealer': False, 'car_dealer_...","{'address_line': '', 'city': {'id': None, 'nam...","{'city_id': None, 'city_name': 'Villa Celina',...","[{'attribute_group_id': 'MAIN', 'attribute_gro...","{'amount': 29333.17, 'currency_id': 'ARS', 'qu...",None,True,None,[],KOEQ19297,{'id': 35713325.0}
4,MLA1377070100,Tv Led 4k 50 Noblex Dk50x7500 Uhd Smart Netf...,new,905088-MLA52385764775_112022,MLA20005533,gold_special,https://www.mercadolibre.com.ar/tv-led-4k-50-n...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_905088-MLA52385764...,ARS,5,113999.0,169999.0,None,5,100,NaN,None,True,True,"[ahora-paid-by-buyer, extended_warranty_eligib...","{'free_shipping': True, 'logistic_type': 'fulf...",2043-03-16T04:00:00.000Z,"{'_': False, 'car_dealer': False, 'car_dealer_...","{'address_line': '', 'city': {'id': 'TUx

# campos importantes
## categoricos:
- listing_type_id (categoria cliente?)
- buying_mode (modo compra)
- site_id (jerarquia sitio web)
- category_id (categoria producto?)
- domain_id (?)
- currency_id (tipo moneda)
- official_store_id (que son tiendas oficiales?)



## numericos:
- currency_id
- order_backend (orden productos web) -> son los primeros productos los mas vendidos? quienes tienen mas stock?
- price (precio a la fecha de consulta)
- original_price (con esta columna != null vemos descuentos)
- sale_price (?)
- sold_quantity (articulos vendidos)
- available_quantity (articulos disponibles)

## booleanos:
- accepts_mercadopago

## timestamp:
- stop_time (tiempo finalización publicación)

## diccionarios:
- tags (otras descripciones - jerarquias a investigar)
- seller (otras jerarquias a investigar)
- seller_address (direccion para jerarquia paises, ej: ciudad)
- address (idem anterior)
- attributes (investigar)
- installments (investigar)

--notas:
unicos por id ? (MLA1386115404)
catalog_product_id (MLA21728449)
categoria cliente listing_type_id? 

- Analisis comparativo venta categorías por países (análisis demanda meli tecnologia)
- Analisis precios medios por países (llevar a dolar) inflacion?
- Análisis tags (wordcloud)
- Análisis penetración mercado pago por regiones / tipos producto / etc
- Le importa a meli el stock de sus clientes?
- Podemos saber que cliente es fraudulentro? -> Ítems con pérdida de exposición en https://developers.mercadolibre.com.ar/es_ar/items-y-busquedas




In [2]:
len(pdf)

1050